In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

df_combats = pd.read_csv("Datos/combats.csv")
df_combats = df_combats.sample(frac = 1)
df_pokemon = pd.read_csv("Datos/pokemon2.csv")
df_pokemon = df_pokemon.sample(frac = 1)

In [2]:
# Limpieza y configuración de dataframes
# Se cambia el nombre de ambos IDs y cambiamos el id del ganador por un booleano que indica si gana el primero
df_combats.rename(columns = {'First_pokemon':'ID1', 'Second_pokemon':'ID2', 'Winner': 'isFirstWinner'}, inplace = True)
df_combats.loc[df_combats['isFirstWinner'] == df_combats['ID1'], 'isFirstWinner'] = True
df_combats.loc[df_combats['isFirstWinner'] == df_combats['ID2'], 'isFirstWinner'] = False
df_combats.dropna(inplace=True)

In [3]:
# Unificamos las dos columnas de tipos y creamos una columna por cada tipo que hay
types1 = pd.get_dummies(df_pokemon["Type 1"])
types2 = pd.get_dummies(df_pokemon["Type 2"])
pd.set_option('max_rows', None)
pd.reset_option('max_rows')
types = types1 | types2

df_clean = pd.merge(df_pokemon, types, left_index=True, right_index=True)

df_clean.dropna(thresh=20,inplace=True)
df_clean.drop(["Type 1", 'Type 2'], axis = 1,inplace= True)
df_clean

,#,Name,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
413,414,Mega Metagross,80,145,150,105,110,110,3,False,...,0,0,0,0,0,0,1,0,1,0
600,601,Sewaddle,45,53,70,40,60,42,5,False,...,0,1,0,0,0,0,0,0,0,0
450,451,Luxray,80,120,79,95,79,70,4,False,...,0,0,0,0,0,0,0,0,0,0
448,449,Shinx,45,65,34,40,34,45,4,False,...,0,0,0,0,0,0,0,0,0,0
263,264,Entei,115,115,85,90,75,100,2,True,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,722,Fennekin,40,45,40,62,60,60,6,False,...,0,0,0,0,0,0,0,0,0,0
120,121,Rhydon,105,130,120,45,45,40,1,False,...,0,0,1,0,0,0,0,1,0,0
362,363,Cacnea,50,85,40,85,40,35,3,False,...,0,1,0,0,0,0,0,0,0,0
227,228,Qwilfish,65,95,75,55,55,85,2,False,...,0,0,0,0,0,1,0,0,0,1


In [4]:
# Mapa de correlaciones
corr = round (df_clean.corr(), 3)
corr.style.background_gradient()

,#,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
#,1.000000,0.098000,0.103000,0.095000,0.089000,0.086000,0.012000,0.983000,0.154000,-0.040000,0.099000,0.141000,0.016000,0.050000,0.048000,-0.009000,-0.033000,0.169000,0.044000,-0.047000,0.071000,-0.053000,-0.215000,0.003000,0.025000,0.093000,-0.102000
HP,0.098000,1.000000,0.422000,0.240000,0.362000,0.379000,0.176000,0.059000,0.274000,-0.154000,0.010000,0.138000,-0.061000,-0.007000,0.055000,0.010000,0.031000,-0.062000,-0.046000,0.075000,0.082000,0.112000,-0.076000,0.027000,-0.036000,-0.043000,0.030000
Attack,0.103000,0.422000,1.000000,0.439000,0.396000,0.264000,0.381000,0.051000,0.345000,-0.070000,0.143000,0.213000,-0.075000,-0.123000,0.211000,0.046000,0.014000,-0.016000,-0.063000,0.127000,0.021000,-0.075000,-0.073000,-0.070000,0.101000,0.108000,-0.071000
Defense,0.095000,0.240000,0.439000,1.000000,0.224000,0.511000,0.015000,0.042000,0.246000,-0.028000,-0.024000,0.106000,-0.061000,-0.025000,0.002000,-0.035000,-0.068000,0.061000,-0.007000,0.134000,0.020000,-0.175000,-0.094000,0.011000,0.298000,0.351000,0.006000
Sp. Atk,0.089000,0.362000,0.396000,0.224000,1.000000,0.506000,0.473000,0.036000,0.449000,-0.185000,0.037000,0.194000,0.127000,0.026000,-0.048000,0.182000,0.041000,0.035000,0.001000,-0.107000,0.070000,-0.184000,-0.044000,0.237000,-0.106000,-0.006000,0.026000
Sp. Def,0.086000,0.379000,0.264000,0.511000,0.506000,1.000000,0.259000,0.028000,0.364000,-0.082000,-0.021000,0.139000,0.017000,0.093000,0.014000,0.026000,-0.009000,0.045000,-0.005000,-0.077000,0.060000,-0.112000,-0.047000,0.190000,0.019000,0.103000,-0.025000
Speed,0.012000,0.176000,0.381000,0.015000,0.473000,0.259000,1.000000,-0.023000,0.327000,-0.071000,0.068000,0.123000,0.130000,-0.098000,0.076000,0.073000,0.237000,-0.061000,-0.096000,-0.096000,-0.005000,0.052000,-0.030000,0.108000,-0.165000,-0.098000,-0.049000
Generation,0.983000,0.059000,0.051000,0.042000,0.036000,0.028000,-0.023000,1.000000,0.080000,-0.019000,0.094000,0.102000,0.006000,0.066000,0.051000,0.006000,-0.036000,0.156000,0.073000,-0.051000,0.038000,-0.036000,-0.192000,-0.022000,0.009000,0.079000,-0.105000
Legendary,0.154000,0.274000,0.345000,0.246000,0.449000,0.364000,0.327000,0.080000,1.000000,-0.094000,-0.021000,0.226000,0.018000,-0.005000,-0.006000,0.047000,0.094000,-0.014000,-0.067000,-0.007000,0.041000,-0.086000,-0.086000,0.169000,-0.013000,0.019000,-0.066000
Bug,-0.040000,-0.154000,-0.070000,-0.028000,-0.185000,-0.082000,-0.071000,-0.019000,-0.094000,1.000000,-0.082000,-0.081000,-0.045000,-0.072000,-0.049000,-0.061000,0.065000,-0.059000,-0.034000,-0.064000,-0.070000,-0.120000,0.121000,-0.112000,-0.004000,0.047000,-0.124000


In [5]:
# Igualamos el nombre de la columna del id a ID1
df_clean.rename(columns = {'#':'ID1'}, inplace= True)
#Añadimos todos los campos del 1er pokemon a la tabla
df_combats = df_combats.merge(df_clean, on='ID1')

# Igualamos el nombre de la columna del id a ID2
df_clean.rename(columns = {'ID1':'ID2'}, inplace= True)
#Añadimos todos los campos del 1er pokemon a la tabla
df_combats = df_combats.merge(df_clean, on='ID2')

In [6]:
# Escalamos los datos
from sklearn import preprocessing

# Cambiamos true y false por ints 1 y 0 
df_combats["isFirstWinner"] = df_combats["isFirstWinner"].astype(int)

X_scaled = preprocessing.scale(df_combats.drop(["ID1","ID2","Name_x","Name_y"], axis=1))
df_scaled = pd.DataFrame(X_scaled)
df_scaled

,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
0,-0.944081,0.828149,1.250482,1.446229,-0.393764,-0.432618,-0.621907,-0.800051,-0.298316,-0.317853,...,-0.240924,-0.360406,3.291127,-0.227526,-0.379725,-0.286794,-0.356676,-0.281905,-0.257228,2.306061
1,-0.944081,-0.596074,-1.043759,-0.861341,0.064318,0.928603,-0.553620,1.620898,-0.298316,-0.317853,...,-0.240924,-0.360406,3.291127,-0.227526,-0.379725,-0.286794,-0.356676,-0.281905,-0.257228,2.306061
2,-0.944081,0.036914,0.179837,-0.134299,-0.393764,-0.074402,-0.621907,-0.194814,-0.298316,-0.317853,...,-0.240924,-0.360406,3.291127,-0.227526,-0.379725,-0.286794,-0.356676,-0.281905,-0.257228,2.306061
3,1.059231,1.025958,0.638685,0.339860,1.072096,-0.074402,0.060966,1.015661,-0.298316,-0.317853,...,-0.240924,-0.360406,3.291127,-0.227526,-0.379725,-0.286794,-0.356676,-0.281905,-0.257228,2.306061
4,1.059231,0.036914,-0.126062,0.181807,1.896643,0.283814,2.280305,-0.194814,-0.298316,-0.317853,...,-0.240924,-0.360406,3.291127,-0.227526,-0.379725,-0.286794,-0.356676,-0.281905,-0.257228,2.306061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,-0.944081,0.234723,0.485735,-0.229130,1.591255,0.821138,0.504834,0.410423,-0.298316,-0.317853,...,-0.240924,-0.360406,-0.303847,-0.227526,-0.379725,-0.286794,-0.356676,-0.281905,-0.257228,-0.433640
49996,1.059231,1.223766,0.638685,0.814018,0.827786,1.000246,1.085276,0.410423,-0.298316,-0.317853,...,-0.240924,-0.360406,-0.303847,-0.227526,-0.379725,-0.286794,-0.356676,-0.281905,-0.257228,-0.433640
49997,1.059231,-0.160894,0.332786,-0.292351,-0.363225,-0.396797,1.085276,-1.405288,-0.298316,-0.317853,...,-0.240924,-0.360406,-0.303847,-0.227526,-0.379725,-0.286794,-0.356676,-0.281905,-0.257228,-0.433640
49998,-0.944081,-0.358703,-0.890809,0.814018,0.369705,-0.074402,-0.109752,-0.194814,-0.298316,-0.317853,...,-0.240924,-0.360406,-0.303847,-0.227526,-0.379725,-0.286794,-0.356676,-0.281905,-0.257228,-0.433640


In [7]:
# Variables para entrenar el modelo 
Y = df_scaled[0] > 0 # Combates en los que el primer pokemon gana
X = pd.get_dummies(df_scaled.drop([0], axis=1)) # Resto de columnas del dataframe menos la columna 'isFirstWinner'

In [8]:
# Ajustar el modelo Bermoulli
clf = BernoulliNB()
clf.fit(X, Y)
Y_pred = clf.predict(X)
np.mean(Y == Y_pred)

0.75402

In [9]:
# Ajustar el modelo Gausiiano
clf = GaussianNB()
clf.fit(X, Y)
Y_pred = clf.predict(X)
np.mean(Y == Y_pred)

0.74094